### GeoJSON과 Choropleth Map - 서울특별시.

In [ ]:
import json                                       # json = JavaScript Object Notation.
import pandas as pd
import folium
from folium import plugins
from ipywidgets import interact                   # Ineractive 시각화 기능.
from sklearn.preprocessing import MinMaxScaler    # scikit-learn 라이브러리의 전처리 기능.

In [ ]:
# 구글 드라이브 마운트.
from google.colab import drive
drive.mount('/content/drive')  # 절차를 따라서 한다.

In [ ]:
# 경로 이동.
%cd "/content/drive/MyDrive/GwangjuAI/visualization/notebook"

#### 1. GeoJSON 파일을 읽어와서 탐색한다:

In [ ]:
# 지리정보를 담은 GeoJSON 파일을 읽어온다.
f = open('../data/geo_seoul.json', encoding='utf8')
geo_seoul = json.load(f)
f.close()

In [ ]:
# geo_seoul을 한번 훑어본다.
geo_seoul

In [ ]:
# 구 이름을 가져와서 본다.
my_kus = []
for a_dict in geo_seoul['features']:                            # List of dictionary objects.
    my_kus.append(a_dict['id'])                                 # id = 구.
print(my_kus)

In [ ]:
# Geometry의 type을 출력해 본다.
# Polygon 유형만 있다!
my_geo_types  = []
for a_dict in geo_seoul['features']:                            # List of dictionary objects.
    my_geo_types.append(a_dict['geometry']['type'])             
print(my_geo_types)

In [ ]:
# 구의 경계 좌표를 가져온다.
my_coords =[]
for a_dict in geo_seoul['features']:                            # List of dictionary objects.
    my_coords.append(a_dict['geometry']['coordinates'][0])      # 구의 경계선을 가져와 본다.
    
# 한 개만 출력해 보는데 [경도, 위도]로 입력되어 있는 것을 확인할 수 있다.
my_coords[0]

#### 2. Polygon을 사용해서 구의 경계를 출력해 본다:

**NOTE**: Geometry의 type으로 "Polygon"만 있으니, folium.Polygon으로 경계선을 그려본다.

In [ ]:
# 좌표에서 위도와 경도 위치를 서로 교환한다.
my_coords2 = [ [ [a_lat, a_long] for (a_long, a_lat) in a_polygon ] for a_polygon in my_coords ]  

# 다시 한 개만 출력해 보는데 이제는 [위도,경도]로 입력되어 있는 것을 확인할 수 있다.
my_coords2[0]

In [ ]:
# 구의 경계 좌표를 사용해서 Polygon으로 출력해 본다.
my_map=folium.Map(location = [37.55, 127],  tiles="Stamen Toner", zoom_start=10, width='50%', height='80%')
for a_ku, a_coords in zip(my_kus, my_coords2):
    folium.Polygon(locations=a_coords,             
                   fill=True,
                   fill_color='turquoise',
                   fill_opacity=0.3,
                   color="red",
                   weight=1,
                   opacity=0.5,
                   tooltip=a_ku).add_to(my_map)
my_map

#### 3. Choropleth Map 생성:

3.1. 데이터 전처리:

In [ ]:
# 인구 정보 DataFrame을 읽어 온다.
my_data = pd.read_csv("../data/data_seoul_population.csv",header="infer",encoding="utf8")
my_data.head(3)

In [ ]:
# '구'는 index로 들어가야 하며, 수치형 변수는 MinMaxScale 해주어야 한다.
my_columns = ['세대수','인구','남자','여자']
my_scaler = MinMaxScaler()
X_scaled = my_scaler.fit_transform(my_data[my_columns])
my_data_scaled = pd.DataFrame(data=X_scaled, columns = my_columns, index = my_data['구'])
my_data_scaled.head(5)

3.2. Choropleth Map 출력:

In [ ]:
# 인구 데이터로 채색한다.
my_map=folium.Map(location = [37.55, 127],  tiles="Stamen Toner", zoom_start=10, width='50%', height='80%')       
folium.Choropleth(geo_data=geo_seoul,
                  data=my_data_scaled['인구'],   # Series 또는 Dictionary.
                  fill_opacity=0.5,
                  fill_color='BuPu',
                  line_weight=0.5,
                  line_color='blue',
                  line_opacity=0.4,
                  legend_name='Population',
                  key_on='feature.id'            # 또는 key_on='feature.properties.name'
                  ).add_to(my_map)
my_map

In [ ]:
# 세대수 데이터로 채색한다.
my_map=folium.Map(location = [37.55, 127],  tiles="Stamen Toner", zoom_start=10, width='50%', height='80%')
folium.Choropleth(geo_data=geo_seoul,
                  data=my_data_scaled['세대수'],         # Series 또는 Dictionary.
                  fill_opacity=0.5,
                  fill_color='YlGn',
                  line_weight=0.5,
                  line_color='orange',
                  line_opacity=0.5,
                  legend_name='Households',
                  key_on='feature.id' ).add_to(my_map)   # 또는 key_on='feature.properties.name'
my_map

*fill_color로 가능한 값들:* <br>
- Sequential: 'BuGn', 'BuPu','GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu','YlOrBr','YlOrRd'.
- Diverging: 'BrBg', 'PiYG', 'PRGn', 'PuOr', 'RdBu', 'RdGy', 'RdYlBu', 'RdYlGn', 'Spectral'.
- Qualitative: 'Accent', 'Qualitative', 'Dark2', 'Paired', 'Pastel1', 'Pastel2', 'Set1', 'Set2', 'Set3'.

#### 4. Interactive 시각화:

In [ ]:
# Decorator를 사용하는 방법.
my_columns = ['세대수','인구','남자','여자']
my_colors = ['BuGn', 'BuPu','GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu','YlOrBr','YlOrRd']

@interact( data_column = my_columns, color = my_colors, zoom=(8,12) )
def choropleth_map(data_column='세대수', color='BuGn', zoom=10):
    my_map=folium.Map(location = [37.55, 127],  tiles="Stamen Toner", zoom_start=zoom, width='50%', height='80%')
    folium.Choropleth(geo_data=geo_seoul,
                  data=my_data_scaled[data_column],      
                  fill_opacity=0.5,
                  fill_color=color,
                  line_weight=0.5,
                  line_color='orange',
                  line_opacity=0.5,
                  key_on='feature.id' ).add_to(my_map)   # 또는 key_on='feature.properties.name'
    return my_map

In [ ]:
# 또다른 방법.
res = interact(choropleth_map, data_column = my_columns, color = my_colors, zoom=(8,12) )

#### 5. Animation:

In [ ]:
# 준비.
n_features = len(geo_seoul['features'])
my_minutes = [ ("0" + str(i))[-2:] for i in range(n_features+1)]

In [ ]:
# "features"를 다시 구성한다.
my_features = [  {'type':'Feature', 
                   'geometry':a_feature['geometry'], 
                 'properties':{
                 'times': [f'2021-01-01T00:{my_minutes[i]}:00', f'2021-01-01T00:{my_minutes[i+1]}:00'],
                 'style':{ 'color':'orange', 'weight': 0.5 , 'fillColor':'blue', 'fillOpacity': my_data_scaled['세대수'][a_feature['id']]*0.8+0.1}
                 } } for i, a_feature in zip(range(n_features) , geo_seoul['features']) ]
#my_features[0]

In [ ]:
# 출력.
my_map=folium.Map(location = [37.55, 127],  tiles="Stamen Toner", zoom_start=10, width='50%', height='80%')
plugins.TimestampedGeoJson({"type": "FeatureCollection",
                            "features" : my_features}, 
                             period="PT30S").add_to(my_map)                # 주기는 조정 가능!  
my_map